# Create Distance matrix based on municipalities
* for whole network, between all municipalities
* like an adj.matrix, create matrix with distances between dos

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from os.path import join

from scipy import sparse

In [ ]:
### load data files
DM = pd.read_csv('data/DistanceMatrix.csv',index_col=0,header=0)
MF = pd.read_csv('data/matched_and_imputed_doctors.csv')
AM = sparse.load_npz(join('data', 'adj_all_doctors.npz'))

MF['adj_index'] = MF.index

### load doc info with adjacency indices
doc_IDs = MF[['adj_index','gemeinde']]

## Create distance matrix between all doctors (like adj.matrix)
* for all connections write distance between their municipalities

In [ ]:
AM = AM.todense()

AM_upper = np.triu(AM)

ind_row = np.where(AM_upper)[0]
ind_col = np.where(AM_upper)[1]

### dist matrix for docs
DM_docs = np.zeros((np.size(AM,0),np.size(AM,0)))

for i in np.arange(0,len(ind_row),1):
    DM_docs[ind_row[i],ind_col[i]] = DM.loc[doc_IDs.loc[doc_IDs.adj_index == ind_row[i],'gemeinde'].item(),
                                            str(doc_IDs.loc[doc_IDs.adj_index == ind_col[i],'gemeinde'].item())]
    DM_docs[ind_col[i],ind_row[i]] = DM_docs[ind_row[i],ind_col[i]]
    
    if np.mod(i,100000) == 0:
        print(i)
        

DM_docs_sparse = sp.sparse.csc_matrix(DM_docs)

In [ ]:
sp.sparse.save_npz('data/DistanceMatrixDocs.npz',DM_docs_sparse)